In [9]:
print("Hello")

Hello


In [10]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [45]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col, regexp_replace, trim, lower, concat_ws, collect_list, udf
from pyspark.sql.types import FloatType

In [12]:
# Создание SparkSession
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("WordCount") \
    .getOrCreate()

In [15]:
# Проверка сессии
print(spark)

In [17]:
# Чтение текстового файла, где каждая строка файла - это строка DataFrame
df = spark.read.text("hdfs://namenode:8020/pilevin.txt").withColumnRenamed("value", "line") 

In [22]:
# Очистка текста
cleaned_df = df \
    .withColumn("line", trim(col("line"))) \
    .withColumn("line", regexp_replace(col("line"), r"[^a-zA-Zа-яА-Я0-9\s]", "")) \
    .withColumn("line", lower(col("line")))

# Удаление пустых строк
cleaned_df = cleaned_df.filter(col("line") != "")

In [23]:
# Вывод первых нескольких строк очищенного текста
cleaned_df.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|line                                                                                                                                                                                                                                                          |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|transhumanism inc трансгуманизм inc                                                                                                                                                                                                 

In [24]:
# Вывод первых нескольких строк очищенного текста
cleaned_df.show(10)

+--------------------+
|                line|
+--------------------+
|transhumanism inc...|
|виктор олегович п...|
|единственный и не...|
|в будущем богатые...|
|но в банку пустят...|
|кто и как будет б...|
|в связи с нравств...|
|больше интересных...|
|виктор олегович п...|
|   transhumanism inc|
+--------------------+
only showing top 10 rows



In [25]:
# Разделение строк на отдельные слова
words_df = cleaned_df.withColumn("word", explode(split(col("line"), "\s+")))

# Удаление пустых значений (если есть)
words_df = words_df.filter(col("word") != "")

# Подсчет количества каждого слова
word_count_df = words_df.groupBy("word").count().orderBy(col("count").desc())

In [26]:
# Вывод результата
word_count_df.show(10, truncate=False)  # Покажем 10 самых частых слов

+----+-----+
|word|count|
+----+-----+
|и   |798  |
|в   |681  |
|не  |460  |
|на  |399  |
|что |283  |
|с   |260  |
|но  |230  |
|а   |211  |
|как |207  |
|маня|199  |
+----+-----+
only showing top 10 rows



In [27]:
# Получение уникальных слов и сортировка их в алфавитном порядке
unique_words_df = words_df.select("word").distinct().orderBy("word")

# Количество уникальных слов
unique_words_df.count()

8704

In [28]:
# Вывод результата
unique_words_df.show(10, truncate=False)  # Покажем первые 10 слов для примера

+------------+
|word        |
+------------+
|1           |
|187         |
|2           |
|2021        |
|206         |
|206год      |
|3           |
|3dтехнологию|
|7           |
|a           |
+------------+
only showing top 10 rows



In [29]:
# Сохранение списка уникальных слов в файл
unique_words_df.coalesce(1).write.mode("overwrite").text("hdfs://namenode:8020/unique1.txt")

In [ ]:
# Анализ тональности текста

In [31]:
!pip install textblob

     |████████████████████████████████| 636 kB 484 kB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 14.9 MB/s eta 0:00:01
     |████████████████████████████████| 761 kB 17.3 MB/s eta 0:00:01     |█████████████▍                  | 317 kB 17.3 MB/s eta 0:00:01


In [48]:
# Чтение файла из HDFS
df = spark.read.text("hdfs://namenode:8020/pilevin.txt")

In [49]:
# Собираем все строки в один текст
full_text = " ".join(row['value'] for row in df.collect())

# Анализ тональности всего текста
blob = TextBlob(full_text)
print(f"Полярность: {blob.sentiment.polarity}, Субъективность: {blob.sentiment.subjectivity}")

Полярность: -0.6145833333333334, Субъективность: 0.5666666666666667


In [ ]:
"""
Пояснение:
Полярность (Polarity: -0.6145833333333334):
Полярность отражает эмоциональный тон текста и принимает значения от -1 до 1:
-1 означает крайне негативную тональность.
0 означает нейтральную тональность.
1 означает крайне позитивную тональность.
В данном случае значение -0.614 указывает на довольно выраженную негативную тональность текста, то есть текст содержит много негативных слов или фраз.

Субъективность (Subjectivity: 0.5666666666666667):
Субъективность указывает, насколько текст субъективен или объективен:
Значение 0 означает полностью объективный, фактологический текст.
Значение 1 означает полностью субъективный, основанный на мнениях, чувствах и предположениях текст.
"""